# Bandit Problem 

## 什么是多臂赌博机问题

> 假设赌博机有$n$个手臂，玩家$Player_A$每摇一次手臂需要花费一枚代币，当玩家$Player_A$花费一枚代币任意摇一次手臂$i$后，赌博机会随机吐出金币的数量为$r_t(i)$，玩家获得这些金币后继续进行下一次的摇手臂动作。这些金币$r_{t}(i)$表示玩家$Player_A$在进行第$t$次动作时去摇手臂$i$获得的回报，摇一次手臂$i$叫做执行一次动作$a_t(i)$，因为赌博机只有$n$个手臂，因此可执行的动作是有限的$n$个动作$a_t(i),i=1,2,...,n$。如果玩家总共只有$T$个代币，那么玩家最多只有$T$次摇手臂的机会，在$T$次动作执行完之后，把所有回报相加就得到累积回报$R_T = \sum_{t=1}^T r_{t}$。那么很自然地引出多臂赌博机的__基本问题：如何在有限次时间长度$T$的时间序列上去分布这些动作$a_t(i)$，才能获得最大的累积回报$R_T$？__     
> 为了回答该问题，本文首先对基本问题作进一步地数学解释，然后给出任何算法都不能超越的上下界，并介绍当前的一些算法以及这些算法之间的区别，最后给出一个好的赌博机算法应当具备品质，以及如何寻求新的算法。    

## 什么是回报与累积回报 - 随机变量，概率分布，随机过程，什么是随机过程？

> **回报$r_t(i)$**：在第$t$次，玩家摇一次手臂$i$，赌博机随机吐出的金币数量为$r_t(i)$。可见，摇不同手臂，会得到不同的金币数量。对于同一个手臂，在不同的时间摇，也会得到不同数量的金币。那么什么是回报呢？--- 也就是，回报是定义在不同手臂上的一个随机变量，概率分布，或者随机过程？   
>> 如果把摇手臂看作是一次实验，吐出的金币数量即为该次实验结果对应的一个数，那么回报$r_t(i)$就可以看成该实验的一个随机变量。   
   
> **概率的公理化定义**:   
> **随机变量**：给定一个实验，实验的所有结果构成实验空间$S$，$S$的子集构成的域称为事件，并对这些事件赋予概率。如果对实验的每个结果$\xi$指定一个数$\pmb x(\xi)$，于是建立了一个定义在$S$集合上的函数$\pmb x$，该函数的值域为一个数集，如果该函数满足如下两个条件，则称该函数$\pmb x$为随机变量。    
>> 条件I. 对于每个$x$，集合$\{\pmb x \leq x\}$是一个事件。    
>> 条件II. 事件$\{\pmb x = +\infty\}$和事件$\{\pmb x = -\infty\}$的概率等于0。即，$$P\{\pmb x = +\infty\}=0，P\{\pmb x = -\infty\}=0$$  
>> 总之，随机变量$\pmb x$就是满足上面两个条件的，将每个实验结果$\xi$指定一个数$\pmb x(\xi)$的函数$\pmb x$。  
>> 注：符号$\{\pmb x \leq x\}$表示$S$的一个子集，因此也是一个事件，由满足$\pmb x \leq x$的所有结果$\xi$组成。  
  
> **随机变量的概率分布**：
>> 要点：**从随机变量的定义来看，随机变量是定义在一个给定的实验的所有结果上的，其本质是一个实验结果与数之间的对应函数。符号$\{\pmb x \leq x\}$表示满足$\pmb x \leq x$的所有结果$\xi$组成的一个集合，也就表示为一个事件。**   

>> 分布函数： $$ F(x) = P\{\pmb x \leq x\}$$   $$P\{x_1 < \pmb x \leq x_2\} = F(x_2) - F(x_1)$$    
>> 概率密度函数： $$ f(x) = \frac{\mathrm{d} F(x)}{\mathrm{d} x} = \lim_{\Delta x \rightarrow 0} \frac{\mathrm{d} F(x+\Delta x) - F(x)}{\mathrm{d} x} = \lim_{\Delta x \rightarrow 0} \frac{P\{x < \pmb x \leq x + \Delta x \}}{\mathrm{d} x}, f(x) \geq 0 $$   $$F(x) = \int ^x_{-\infty} f(u) {\rm d}u $$
>> 常见的随机变量类型：   
- 连续型随机变量：均匀分布，正态分布，指数分布，beta分布，伽马分布，卡方分布，...   
- 离散型随机变量：伯努利分布，二项分布，泊松分布，负二项分布，...    
>> 注：   
一般认为很多自然现象都服从正态分布，或者近似为正态分布；beta分布常用在贝叶斯估计中，作为先验分布；伯努利分布是最简单的离散型分布，描述任何只有两个结果的实验或只关心某个事件的出现或不出现的两种结果实验，伯努利随机变量便把这两个结果分别映射为两个不同的值，比如0和1，这两个值描述了事件发生与不发生或者表达成功与失败之类的意义，即$P(x=1)=p,P(x=0)=1-p$，成功概率为$p$，失败概率$1-p$。 如果把n次伯努利实验看成一个实验，那么实验的结果就是某个事件发生或不发生组成的序列，比如4次伯努利实验的一个结果1010，如果存在一个随机变量，把这些0和1组成的序列映射成一个数，这个映射规则为1出现的次数，也表示n次伯努利实验中某个事件发生的次数，那么这样的随机变量就叫做二项式随机变量，其值服从二项式分布。    

> **随机变量函数**：   
> **随机序列**：  
> **随机过程**：    

## 执行一次动作是什么？探索或利用，探索是什么，采样，采样是什么？

## 估计的回报的概率分布的可信度(置信度?)为95%时对应的探索次数是多少？区分两个分布需要的最少探索次数是多少？衡量两个分布的相似性？信息量？信息论？样本复杂度？获得的回报的上下界？

## 当前不同算法的区别是什么？- 对估计的最少探索次数的把握程度？

## 赌博机与强化学习的关系是什么？